<font size = 8 color = steelblue> Data Science Project

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
hospital = pd.read_csv("Hospitalisation details.csv")
hospital.head()     

In [ ]:
Medical = pd.read_csv("Medical Examinations.csv")
Medical.head()


In [ ]:
names = pd.read_excel('Names.xlsx')
names.head()

In [ ]:
print(hospital.shape)
print(Medical.shape)
print(names.shape)

### Took data into one place, Merge the data 

In [ ]:
data = pd.merge(hospital,Medical,how = 'outer',on = 'Customer ID')
data = pd.merge(names,data,how='outer', on = 'Customer ID')
data.tail()

In [ ]:
data.shape

### Check for missing values in the dataset

In [ ]:
hospital.isna().sum()

In [ ]:
Medical.isna().sum()

In [ ]:
names.isna().sum()

In [ ]:
data.isna().sum()

In [ ]:
data[data.isnull().any(axis = 1)]

### Columns that contain "?" Count in Data


In [ ]:
question_counts = data[data == '?'].count()
non_zero = question_counts[question_counts != 0]
non_zero

### Total Percent of rows that contain '?' Values

In [ ]:
print(((non_zero.sum() / data.shape[0]) * 100).round(2),'%')

### Column Wise

In [ ]:
trival_val_percent = (non_zero.values / data.shape[0] * 100).round(2)
trival_val_percent = pd.DataFrame(trival_val_percent,index = non_zero.index,columns=['Percent'])
trival_val_percent.sort_values('Percent',ascending=False)

### Data with contain "?" in all columns

In [ ]:
ques_data = []
for col in (non_zero.index):
    display(data[data[col] == '?'])
    l = data[data[col] == '?'].index.to_list()
    ques_data.append(l)

### Droping Null values or "?" values

In [ ]:
data.replace("?",np.nan,inplace=True)
data.dropna(inplace=True)

### Transformation to deal with the nominal and ordinal categorical variables in the dataset

In [ ]:
data.dtypes[data.dtypes == 'object']

In [ ]:
data.year = data.year.astype(int)
data['Customer ID'] = data['Customer ID'].str.removeprefix('Id')
data['Customer ID'] = data['Customer ID'].astype(int)

In [ ]:
data.month.unique()

In [ ]:
data.month.value_counts()

In [ ]:
months = {'Oct': 10, 'Dec': 12, 'Jun': 6, 'Sep': 9, 'Aug': 8, 'Nov': 11, 'Jul': 7}
data['month'] = data['month'].map(months).astype(int)

In [ ]:
data.month.value_counts()

### Drop name column & convert all categorical column exclude StateID column

In [ ]:
data1 = data.drop(columns='name')
col = data1.dtypes[(data.dtypes == 'object') & (data.columns != 'State ID')].index.tolist()
dummy_data = pd.get_dummies(data1,drop_first=True,columns=col)

In [ ]:
dummy_data.head()

### Creating dummy variables with State R1011, R1012, and R1013.

In [ ]:
data['State ID'].value_counts()

In [ ]:
dummy_data = pd.get_dummies(data['State ID'].apply(lambda x: x if x in ['R1011', 'R1012','R1013'] else None))
dummy_data = pd.concat([data,dummy_data],axis = 1)
dummy_data.head()

### To replace string in  NumberOfMajorSurgeries Column.

In [ ]:
dummy_data['NumberOfMajorSurgeries'].value_counts()

In [ ]:
dummy_data['NumberOfMajorSurgeries'] = dummy_data['NumberOfMajorSurgeries'].replace('No major surgery',0).astype(int)
dummy_data['NumberOfMajorSurgeries'].value_counts()

### Age Calculation using year column

In [ ]:
from datetime import datetime, date
datetime.now().year

In [ ]:
data['Age'] = datetime.now().year - data['year']
data['Age']

### To determine Gender of Patient from name column 

In [ ]:
data.name

In [ ]:
data[['firsts','last_name']] = data['name'].str.split('.',expand=True,n= 1)
data[['first_name','gender']] = data['firsts'].str.split(',',expand=True)
data.drop(columns = ['name','firsts'],inplace = True)

In [ ]:
data.gender = data.gender.replace(' Ms','male')
data.gender = data.gender.replace(' Mr','female')
data.gender = data.gender.replace(' Mrs','female')
data.head()

In [ ]:
data.gender.value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Distribution of charges

In [ ]:
data.charges.hist()
plt.show()

In [ ]:
sns.histplot(data.charges,kde = True)
plt.show()

In [ ]:
sns.boxplot(data.charges)
plt.show()

### Distribution of charges among hospital tiers

In [ ]:
plt.figure(figsize=(10,7))
sns.swarmplot(x = data['Hospital tier'],y = data.charges,size = 2,hue = data['Hospital tier'])
plt.show()

### Distribution across gender and tiers of hospitals

In [ ]:
sns.histplot(x = data['Hospital tier'],y = data.charges,hue = data.gender)
plt.show()

In [ ]:
sns.barplot(x = data['Hospital tier'],y = data.charges,hue = data.gender)
sns.lineplot(x = data['Hospital tier'],y = data.charges,hue = data.gender)
plt.show()

### Radar chart to showcase the median hospitalization cost for each tier of hospitals

In [ ]:
data.groupby('Hospital tier')['charges'].median().values.tolist()

In [ ]:
data['Hospital tier'].value_counts().index.sort_values().tolist()

In [ ]:
labels = data['Hospital tier'].value_counts().index.sort_values().tolist()
values = data.groupby('Hospital tier')['charges'].median().values.tolist()

values += values[:1]  

angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]  

fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))
plt.xticks(angles[:-1], labels)

ax.plot(angles, values, linewidth=1, linestyle='solid')
ax.fill(angles, values, color='blue', alpha=0.25)

plt.title('Radar Chart', size=20, color='darkblue', y=1.1)
plt.show()      

### Frequency table and a stacked bar chart to visualize the count of people in the different tiers of cities and hospitals

In [ ]:
data['City tier'].value_counts()

In [ ]:
data['Hospital tier'].value_counts()

In [ ]:
freq_table = pd.crosstab(index = data['City tier'],columns=data['Hospital tier'])
freq_table

In [ ]:
frequency_table = pd.pivot_table(data,index = ['City tier'],columns = ['Hospital tier'],values=['charges']).round(2)
frequency_table

In [ ]:
freq_table.plot(kind='bar')
plt.show()

In [ ]:
y1 = freq_table.values[0].tolist()
y2 = freq_table.values[1].tolist()
y3 = freq_table.values[2].tolist()

### Stack bar Chart 

In [ ]:
x = freq_table.index.tolist()
plt.bar(x,y1,label = 'hospital tier 1')
plt.bar(x,y2,bottom=y1,label = 'hospital tier 2')
plt.bar(x,y3,bottom=np.array(y1) + np.array(y2),label = 'hospital tier 3')
plt.xlabel('City Tier')
plt.ylabel('Count of people')
plt.title('Stack Bar Chart for Count of People')
plt.legend()
plt.show()

#### Hypothesis testing for average hospitalization costs for the three types of hospitals are not significantly different

In [ ]:
data.groupby('Hospital tier')['charges'].mean()

In [ ]:
hospital1 = data.loc[data['Hospital tier'] == 'tier - 1','charges']
hospital2 = data.loc[data['Hospital tier'] == 'tier - 2','charges']
hospital3 = data.loc[data['Hospital tier'] == 'tier - 3','charges']

In [ ]:
alpha = 0.05
def conclusion(st,p_val,alpha = 0.05):
    print(f"Stats- {st:0.4f} Pvalue: {p_val:0.4f}")
    if p_val < alpha:
        print(f"Reject the Null at alpha = {alpha*100}%.")
    else:
        print(f"Fail to reject the Null at alpha = {alpha*100}%.")

In [ ]:
import scipy.stats as stats
from scipy.stats import f_oneway
f,p = f_oneway(hospital1,hospital2,hospital3)
conclusion(f,p)

#### Hypothesis Testing for average hospitalization costs for the three types of cities are not significantly different.

In [ ]:
data.groupby('City tier')['charges'].mean()

In [ ]:
city1 = data.loc[data['City tier'] == 'tier - 1','charges']
city2 = data.loc[data['City tier'] == 'tier - 2','charges']
city3 = data.loc[data['City tier'] == 'tier - 3','charges']

In [ ]:
f,p = f_oneway(city1,city2,city3)
conclusion(f,p)

#### Hypothesis testing for average hospitalization cost for smokers is not significantly different from the average cost for nonsmokers

In [ ]:
data.groupby('smoker')['charges'].mean()

In [ ]:
sample_1 = data.loc[data.smoker == 'yes','charges']
sample_2 = data.loc[data.smoker == 'No','charges']
t, p = stats.ttest_ind(sample_1,sample_2,alternative='greater') 
conclusion(t,p)

#### Hypothesis testing for Smoking and heart issues are independent.

In [ ]:
heart_issue_smoker = data[(data['Heart Issues'] == 'yes') & (data['smoker'] == 'yes')]['charges']
heart_issue_no_smoker = data[(data['Heart Issues'] == 'yes') & (data['smoker'] == 'No')]['charges']
no_heart_issue_smoker = data[(data['Heart Issues'] == 'No') & (data['smoker'] == 'yes')]['charges']
no_heart_issue_no_smoker = data[(data['Heart Issues'] == 'No') & (data['smoker'] == 'No')]['charges']

In [ ]:
a = len(heart_issue_smoker)
b = len(heart_issue_no_smoker)
c = len(no_heart_issue_smoker)
d = len(no_heart_issue_no_smoker)
contigency_table = [[a,b],[c,d]]
contigency_table

In [ ]:
from scipy.stats import chi2_contingency
chi2, p_value, dof, expected = chi2_contingency(contigency_table)
print(chi2)
print(p_value)
print(dof)      
print(expected)
alpha = 0.05
if p_value <= alpha:
    print(f"Reject null at stats- {chi2}: ")
else:
    print(f"Fail to reject the null at {alpha*100:.1f}%")    

In [ ]:
conclusion(chi2,p_value)